In [1]:
import json
import math
import torch
from torchvision.models.detection import fasterrcnn_resnet50_fpn as faster_rcnn
import torchvision.models.detection as d
from torchvision import transforms
from PIL import Image
import os
from torch.optim import Adam
import numpy as np
from torch.utils.data import Dataset, DataLoader
import pickle
from tqdm.notebook import tqdm

In [43]:
with open ('labels_train.json', 'r', encoding='utf-8') as file:
    labels = json.load(file)
labels[0]

{'id': 28,
 'annotations': [{'id': 162,
   'completed_by': 1,
   'result': [{'original_width': 47,
     'original_height': 50,
     'image_rotation': 0,
     'value': {'x': 28.528270545319657,
      'y': 12.97576176416152,
      'width': 47.85387317279425,
      'height': 22.491320391213307,
      'rotation': 0,
      'rectanglelabels': ['Airplane']},
     'id': 'PPivq2L9kh',
     'from_name': 'label',
     'to_name': 'image',
     'type': 'rectanglelabels',
     'origin': 'manual'}],
   'was_cancelled': False,
   'ground_truth': False,
   'created_at': '2023-11-09T07:56:58.821955Z',
   'updated_at': '2023-11-09T07:56:58.822004Z',
   'draft_created_at': '2023-11-08T12:40:04.847593Z',
   'lead_time': 32.876,
   'prediction': {},
   'result_count': 0,
   'unique_id': '8a7257a6-d118-41fe-a31e-62abeed35a14',
   'import_id': None,
   'last_action': None,
   'task': 28,
   'project': 2,
   'updated_by': 1,
   'parent_prediction': None,
   'parent_annotation': None,
   'last_created_by': None

In [44]:
# sprawdamy max_widht i max_height iterujemy po zdjeciach, 
# robimy carny padding symetryccznie i przesuawmy dla danego zdjecia ospowiednio o tyle bb
max_height = 0
max_width = 0

for path in os.listdir('./photos/train/wybrane'):
    img = Image.open(os.path.join('./photos/train/wybrane', path))
    w, h = img.size
    print(w, h)
    max_width = max(w, max_width)
    max_height = max(h, max_height)

max_height, max_width



227 223


(223, 227)

In [45]:
def add_margin(pil_img, top, right, bottom, left, color):
    width, height = pil_img.size
    new_width = width + right + left
    new_height = height + top + bottom
    result = Image.new(pil_img.mode, (new_width, new_height), color)
    result.paste(pil_img, (left, top))
    return result

In [46]:
def find_dict_by_id(list_of_dicts, target_id):
    for dictionary in list_of_dicts:
        if 'id' in dictionary and dictionary['id'] == target_id:
            return dictionary
    return None

In [47]:
compose = transforms.Compose([
    transforms.PILToTensor(),
    transforms.ConvertImageDtype(torch.float)
])

tensors_image = []
boxes_list = []
labels_list = []
from PIL import ImageDraw
for path in os.listdir('./photos/train/wybrane'):
    img = Image.open(os.path.join('./photos/train/wybrane', path))
    w, h = img.size

    img = add_margin(img, (max_height - h)//2, (max_width - w)//2, math.ceil((max_height - h)/2), math.ceil((max_width - w)/2), (0,0,0))
    tensors_image.append(compose(img))

    id_ = int(path.split('.')[0])
    # id_ = 28
    # print(id_)
    label = find_dict_by_id(labels, id_)
    coordinates = label['annotations'][0]['result'][0]['value']
    # print(coordinates)
    boxes_list.append([coordinates['x'] + (max_width - w)//2, 
                        coordinates['y'] + (max_height - h)//2, 
                        coordinates['x'] + (max_width - w)//2 + coordinates['width'],
                        coordinates['y'] + (max_height - h)//2 + coordinates['height']])
    labels_list.append(1)

   
    draw = ImageDraw.Draw(img)
    draw.rectangle((coordinates['x'] + (max_width - w)//2, 
                        coordinates['y'] + (max_height - h)//2, 
                        coordinates['x'] + (max_width - w)//2 + coordinates['width'],
                        coordinates['y'] + (max_height - h)//2 + coordinates['height']), outline='red', width=2)
    print(boxes_list[0])
    img.show()
    break
# print(type(boxes_list))

[70.21134158727389, 45.20933521923621, 126.19947784583366, 120.5106082036775]


In [48]:
y_train_boxes = torch.tensor(boxes_list, dtype=torch.float)
y_train_labels = torch.tensor(labels_list, dtype=torch.int64)

y_train = {'boxes': y_train_boxes, 'labels': y_train_labels}
x_train = tensors_image
y_train
# type(y_train)
# y_train.shape
# iteruje po wszystkich sciezkach image.open(obraz) i potem compose(image) arr z tych tensowrow powstalych z compose
# torch stack czy jakos tak

{'boxes': tensor([[ 70.2113,  45.2093, 126.1995, 120.5106]]),
 'labels': tensor([1])}

In [49]:
model = faster_rcnn(weights=d.FasterRCNN_ResNet50_FPN_Weights.DEFAULT, min_size= 200, max_size=300)

In [50]:
class ImageDataset(Dataset):
    def __init__(self, images, labels):
        self.images = images
        self.labels = labels

    def __len__(self):
      return len(self.images)
    
    def __getitem__(self, idx):
      return self.images[idx], {'boxes': self.labels['boxes'][idx], 'labels': self.labels['labels'][idx]}

In [51]:
dataLoader = DataLoader(ImageDataset(x_train, y_train), batch_size=10)

In [52]:
y_train['boxes'][0]

tensor([ 70.2113,  45.2093, 126.1995, 120.5106])

In [53]:
optimiser = Adam(model.parameters())
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
print(device)
model = model.to(device)

# torch.DataLoader - zwraca w bachach zdjecia i bb 

def train(model, optimiser, data, epochs=20):
    loss_epoch_mean = []
    for epoch in tqdm(range(epochs)):
        loss_list = np.array([])
        for x, y in data:
            x = x.to(device)
            optimiser.zero_grad()
            y_list = []
            a, _ = y['boxes'].shape
            for i in range(a):
                y_list.append({'boxes': y['boxes'][i].reshape(1, -1).to(device), 'labels': torch.tensor([1]).to(device)})
            loss_dict = model(x, y_list)
            losses = sum(loss for loss in loss_dict.values())
            losses.backward()
            optimiser.step()
            loss_list = np.append(loss_list, losses.item())
        epoch_mean = np.mean(loss_list)
        loss_epoch_mean.append(epoch_mean)
        print(epoch)
    return model, loss_epoch_mean

cuda


In [57]:
model, loss_epoch_mean = train(model, optimiser, dataLoader, 500)

  0%|          | 0/500 [00:00<?, ?it/s]

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

In [58]:
type(model)

torchvision.models.detection.faster_rcnn.FasterRCNN

In [56]:
with open('model_maly.p', 'wb') as f:
  pickle.dump(model, f)

In [22]:
loss_epoch_mean

[0.9639641046524048, 0.9004853963851929]

In [18]:
from .utils import predict, load_model
MODEL = load_model('./model/model.p')

ImportError: attempted relative import with no known parent package